In [144]:
import pandas as pd
import numpy as np

import chart_studio
import chart_studio.plotly as py

import plotly.express as px

In [145]:
username = 'leguela' # your username
api_key = 'dUoZ8GPRarmVDovaJE1F' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 300
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1- Get total vaccination rate over time:

In [3]:
cdc_fr = pd.read_csv('../data/vaccines_numbers/France.csv')
cdc_us = pd.read_csv('../data/vaccines_numbers/United States.csv')

**NB: population france = 68147687 and population USA = 331002647**

In [4]:
# get the percentage of the total population vaccinated
cdc_fr['perc_tot_vac'] = cdc_fr['total_vaccinations'] * 100 / 68147687
cdc_us['perc_tot_vac'] = cdc_us['total_vaccinations'] * 100 / 331002647

In [5]:
# get the percentage of the population vaccinated
cdc_fr['perc_vac'] = cdc_fr['people_vaccinated'] * 100 / 68147687
cdc_us['perc_vac'] = cdc_us['people_vaccinated'] * 100 / 331002647

In [6]:
# get the percentage of the population fully vaccinated
cdc_fr['perc_full_vac'] = cdc_fr['people_fully_vaccinated'] * 100 / 68147687
cdc_us['perc_full_vac'] = cdc_us['people_fully_vaccinated'] * 100 / 331002647

In [7]:
# concat the two datasets
cdc_us_fr = pd.concat([cdc_fr, cdc_us], ignore_index=True)

In [8]:
# change "date" object type to datetime
cdc_us_fr['date'] = cdc_us_fr['date'].astype('Datetime64')

In [146]:
fig1 = px.line(cdc_us_fr, x="date", y="perc_vac", color='location', 
              title= "Share of people who received at least one dose of COVID-19 vaccine",
             labels={
                 "perc_vac": "",
                 "date": "",
                 "location": "Country"
             },
            color_discrete_map={ # replaces default color mapping by value
                "France": "#17becf", "United States": "#d62728"
            },
              template="simple_white" 
             )
fig1.update_yaxes(ticksuffix="%", showgrid=True, showline=False, tickwidth=0, tickcolor='white')
fig1.update_xaxes(showline=True, zeroline=True)
fig1.update(layout_showlegend=False)
fig1.add_annotation(text="France", x="2021-05-29", y=39, arrowhead=1, showarrow=False)
fig1.add_annotation(text="United States", x="2021-05-29", y=52, arrowhead=1, showarrow=False)

fig1.show()

In [160]:
py.plot(fig1, filename = 'covid_vaccination_us_fr_time', auto_open=False)

'https://plotly.com/~leguela/24/'

In [148]:
fig2 = px.line(cdc_us_fr, x="date", y="perc_full_vac", color='location',
              title= "Share of people who received two doses of COVID-19 vaccine",
             labels={
                 "perc_full_vac": "",
                 "date": "",
                 "location": "Country"
             },
            color_discrete_map={ # replaces default color mapping by value
                "France": "#17becf", "United States": "#d62728"
            },
              template="simple_white" 
             )
fig2.update_yaxes(ticksuffix="%", showgrid=True, showline=False, tickwidth=0, tickcolor='white')
fig2.update_xaxes(showline=True, zeroline=True)

fig2.update(layout_showlegend=False)
fig2.add_annotation(text="France", x="2021-05-29", y=18, arrowhead=1, showarrow=False)
fig2.add_annotation(text="United States", x="2021-05-29", y=42, arrowhead=1, showarrow=False)

fig2.show()

In [159]:
py.plot(fig2, filename = 'covid_vaccination_us_two_doses', auto_open=False)

'https://plotly.com/~leguela/29/'

## 2- Get vaccination rate per vaccine per country:

### 1-USA:

In [11]:
vaccination_us = pd.read_csv('../data/vaccines_numbers/COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv')

In [12]:
# lowecase columns names
vaccination_us.columns = vaccination_us.columns.str.lower()

In [13]:
# change "date" object type to datetime
vaccination_us['date'] = vaccination_us['date'].astype('Datetime64')

In [14]:
# Filter to last date and keep few columns
vaccination_us = vaccination_us[vaccination_us.date == '05/31/2021']
vaccination_us = vaccination_us[['date', 'location', 'distributed', 'distributed_janssen', 'distributed_moderna', 'distributed_pfizer', 'distributed_unk_manuf', 'administered', 'administered_janssen', 'administered_moderna', 'administered_pfizer', 'administered_unk_manuf', 'administered_dose1_pop_pct', 'series_complete_pop_pct']]

In [15]:
north_states = ['NY', 'PA', 'NJ', 'DE', 'VT', 'NH', 'RI', 'MA', 'CT']
vac_north = vaccination_us[vaccination_us['location'].isin(north_states)]

south_states = ['TN', 'AL', 'MS', 'GA', 'LA', 'NC', 'SC', 'AR']
vac_south = vaccination_us[vaccination_us['location'].isin(south_states)]

In [123]:
south_distri = vac_south[['distributed_pfizer', 'distributed_moderna', 'distributed_janssen']].rename(columns={'distributed_pfizer':'Pfizer', 'distributed_moderna':'Moderna', 'distributed_janssen':'Janssen'})
south_distri['status'] = 'distribution'
south_distri
south_admin = vac_south[['administered_pfizer', 'administered_moderna', 'administered_janssen']].rename(columns={'administered_pfizer':'Pfizer', 'administered_moderna':'Moderna', 'administered_janssen':'Janssen'})
south_admin['status'] = 'vaccinated'

In [124]:
north_distri = vac_north[['distributed', 'distributed_pfizer', 'distributed_moderna', 'distributed_janssen', 'distributed_unk_manuf']].rename(columns={'distributed_pfizer':'Pfizer', 'distributed_moderna':'Moderna', 'distributed_janssen':'Janssen'})
north_distri['status'] = 'distribution'
north_admin = vac_north[['administered', 'administered_janssen', 'administered_moderna', 'administered_pfizer', 'administered_unk_manuf']].rename(columns={'administered_pfizer':'Pfizer', 'administered_moderna':'Moderna', 'administered_janssen':'Janssen'})
north_admin['status'] = 'vaccinated'

In [125]:
south_all = pd.concat([south_distri, south_admin], ignore_index=True)
north_all = pd.concat([north_distri, north_admin], ignore_index=True)

In [129]:
north_all_gp = north_all.groupby('status')['Pfizer', 'Moderna', 'Janssen'].sum().reset_index()
south_all_gp = south_all.groupby('status')['Pfizer', 'Moderna', 'Janssen'].sum().reset_index()

/Users/armelleleguelte/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/Users/armelleleguelte/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [130]:
north_all_gp = pd.melt(north_all_gp, id_vars=['status'], value_vars=['Pfizer', 'Moderna', 'Janssen'], var_name='vaccine', value_name='sum')
south_all_gp = pd.melt(south_all_gp, id_vars=['status'], value_vars=['Pfizer', 'Moderna', 'Janssen'], var_name='vaccine', value_name='sum')

In [164]:
fig4 = px.bar(north_all_gp, x='vaccine', y='sum', color ='status', barmode='group',title= 'Cumulative COVID-19 doses received and injected in Northeast States',
            labels={
                 "sum": "",
                 "vaccine": "",
                 "status": ""
             },
            color_discrete_map={ # replaces default color mapping by value
                "distribution": "#17becf", "vaccinated": "#d62728"
            },
                        category_orders={"type_de_vaccin": ["Pfizer", "Moderna", "Janssen"]
                        },
             template="simple_white"
            )
fig4.update_yaxes(showgrid=True, showline=False, tickwidth=0, tickcolor='white')
fig4.update_xaxes(showline=True, zeroline=True)
            
fig4.show()

In [165]:
py.plot(fig4, filename = 'covid_vaccination_northeast', auto_open=False)

'https://plotly.com/~leguela/33/'

In [166]:
fig5 = px.bar(south_all_gp, x='vaccine', y='sum', color ='status', barmode='group',title= 'Cumulative COVID-19 doses received and injected in Southern States',
            labels={
                 "sum": "",
                 "vaccine": "",
                 "status": ""
             },
            color_discrete_map={ # replaces default color mapping by value
                "distribution": "#17becf", "vaccinated": "#d62728"
            },
                        category_orders={"type_de_vaccin": ["Pfizer", "Moderna","AstraZeneca", "Janssen"]
                        },
             template="simple_white"
            )
fig5.update_yaxes(showgrid=True, showline=False, tickwidth=0, tickcolor='white')
fig5.update_xaxes(showline=True, zeroline=True)
            
fig5.show()

In [167]:
py.plot(fig5, filename = 'covid_vaccination_southeast', auto_open=False)

'https://plotly.com/~leguela/35/'

In [151]:
south_pop_perc = vac_south[['administered_dose1_pop_pct', 'series_complete_pop_pct']]
north_pop_perc = vac_north[['administered_dose1_pop_pct', 'series_complete_pop_pct']]

In [152]:
south_tot_pop_perc = (south_pop_perc.sum()/8).to_frame().reset_index().rename(columns={'index': 'vaccinated', 0: 'percentage'})
south_tot_pop_perc['states'] = 'Southeast'

In [153]:
north_tot_pop_perc = (north_pop_perc.sum()/9).to_frame().reset_index().rename(columns={'index': 'vaccinated', 0: 'percentage'})
north_tot_pop_perc['states'] = 'Northeast'

In [154]:
pop_perc_all_gp =  pd.concat([south_tot_pop_perc, north_tot_pop_perc], ignore_index=True)

In [155]:
pop_perc_all_gp

,vaccinated,percentage,states
0,administered_dose1_pop_pct,38.562500,Southeast
1,series_complete_pop_pct,31.525000,Southeast
2,administered_dose1_pop_pct,61.144444,Northeast
3,series_complete_pop_pct,49.577778,Northeast


In [161]:
pop_perc_all_gp['vaccinated'] = pop_perc_all_gp['vaccinated'].replace('administered_dose1_pop_pct', 'Only one dose')
pop_perc_all_gp['vaccinated'] = pop_perc_all_gp['vaccinated'].replace('series_complete_pop_pct', 'Fully vaccinated')

In [162]:
fig3 = px.bar(pop_perc_all_gp, x='vaccinated', y='percentage',  color ='states', barmode='group', title='Percentage of people half and fully vaccinated in the United States',
            labels={
                 "percentage": "",
                 "vaccinated": "",
                 "states": "",
                
             },
            color_discrete_map={ # replaces default color mapping by value
                "Southeast": "#17becf", "Northeast": "#d62728"
            },
            category_orders={"states": ["Northeast", "Southeast"]
                        },
             template="simple_white"
            )
fig3.update_yaxes(ticksuffix="%", showgrid=True, showline=False, tickwidth=0, tickcolor='white')
fig3.update_xaxes(showline=True, zeroline=True)
fig3.show()

In [163]:
py.plot(fig3, filename = 'perc_vaccination_us', auto_open=False)

'https://plotly.com/~leguela/27/'

### 2- France:

In [74]:
livraison_fr = pd.read_csv('../data/vaccines_numbers/livraisons-realisees-cumul-nat.csv')

In [75]:
livraison_fr = livraison_fr[livraison_fr.date_fin_semaine == '2021-05-30']
livraison_fr = livraison_fr[['type_de_vaccin', 'nb_doses']]
livraison_fr['status'] = 'distribution'

In [139]:
data = {'type_de_vaccin': ['Pfizer', 'Moderna', 'AstraZeneca', 'Janssen'],
             'nb_doses':[21848732, 2555043, 4364726, 156333],
            }
vaccin_fr = pd.DataFrame(data, columns = ['type_de_vaccin', 'nb_doses'])
vaccin_fr['status'] = 'vaccinated'

In [140]:
tot_vaccin_fr =  pd.concat([livraison_fr, vaccin_fr], ignore_index=True)

In [168]:
fig6 = px.bar(tot_vaccin_fr, x='type_de_vaccin', y='nb_doses', color='status', barmode='group', title='Cumulative COVID-19 doses received and injected in France',
             labels={
                 "nb_doses": "",
                 "type_de_vaccin": "",
                 "status": ""
             },
            color_discrete_map={ # replaces default color mapping by value
                "distribution": "#17becf", "vaccinated": "#d62728"
            },
                        category_orders={"type_de_vaccin": ["Pfizer", "Moderna","AstraZeneca", "Janssen"]
                        },
             template="simple_white"
            )
fig6.update_yaxes(showgrid=True, showline=False, tickwidth=0, tickcolor='white')
fig6.update_xaxes(showline=True, zeroline=True)

fig6.show()

In [169]:
py.plot(fig6, filename = 'covid_vaccination_france', auto_open=False)

'https://plotly.com/~leguela/37/'